In [1]:
import sympy as sp

# Define symbolic variables for Euler angles and tensor coefficients
alpha, beta, gamma = sp.symbols('alpha beta gamma', real=True)
iso, delta_xx, delta_yy, delta = sp.symbols('iso delta_xx delta_yy delta', real=True)

# Define the original coefficients T_{2m}
T_2m = {
    -2: 0.5 * (delta_xx - delta_yy),
    -1: 0,
    0: sp.sqrt(3/2) * delta,
    1: 0,
    2: 0.5 * (delta_xx - delta_yy)
}

# Reduced Wigner d-matrix elements for l=2
d_2 = sp.Matrix([
    [0.25 * (1 + sp.cos(beta))**2, 0.5 * sp.sin(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/4 * sp.sin(beta)**2, 
     0.5 * sp.sin(beta) * (1 - sp.cos(beta)), 0.25 * (1 - sp.cos(beta))**2],
    [-0.5 * sp.sin(beta) * (1 + sp.cos(beta)), sp.cos(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), 
     sp.cos(beta) * (1 - sp.cos(beta)), -0.5 * sp.sin(beta) * (1 - sp.cos(beta))],
    [sp.sqrt(6)/4 * sp.sin(beta)**2, -sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), 0.5 * (3 * sp.cos(beta)**2 - 1), 
     sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), sp.sqrt(6)/4 * sp.sin(beta)**2],
    [-0.5 * sp.sin(beta) * (1 + sp.cos(beta)), -sp.cos(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), 
     -sp.cos(beta) * (1 - sp.cos(beta)), 0.5 * sp.sin(beta) * (1 - sp.cos(beta))],
    [0.25 * (1 + sp.cos(beta))**2, -0.5 * sp.sin(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/4 * sp.sin(beta)**2, 
     -0.5 * sp.sin(beta) * (1 - sp.cos(beta)), 0.25 * (1 - sp.cos(beta))**2]
])

# Construct the full Wigner D-matrix for l=2
m_values = [-2, -1, 0, 1, 2]
D_2 = sp.zeros(5, 5)
for i, m1 in enumerate(m_values):
    for j, m2 in enumerate(m_values):
        D_2[i, j] = sp.exp(-sp.I * m1 * alpha) * d_2[i, j] * sp.exp(-sp.I * m2 * gamma)

# Compute the transformed coefficients T'_{2m}
T_2m_prime = sp.Matrix([0, 0, 0, 0, 0])  # Initialize
for i, m in enumerate(m_values):
    T_2m_prime[i] = sum(D_2[i, j] * T_2m[m_values[j]] for j in range(5))

# Simplify the resulting coefficients
T_2m_prime = T_2m_prime.simplify()
T_2m_prime


In [3]:
T_2m_prime